In [2]:
%cd patches

C:\DATA\Crunch\neuron\patches


In [1]:
import sys
sys.path.append('../')
from VQVAE import VQVAE,VQVAEDataGenerator


In [4]:
from VQVAE import VQVAE,VQVAEDataGenerator
import os
import glob
import random
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Vector Quantizer
from torch.utils.tensorboard import SummaryWriter
import csv
import os

In [5]:
tensor_folder = f'E:/Projects/Gene_expression/Crunch/patches/20'
dataset = VQVAEDataGenerator(tensor_folder)

In [7]:
from torchvision.transforms import ToPILImage

dataset[1].shape
to_pil = ToPILImage()
pil_image = to_pil(dataset[1])

# Display the image (if running locally)
pil_image.show()

In [6]:
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False)

In [7]:
vqvae = VQVAE(image_size=20, latent_dim=16, num_embeddings=1024)
checkpoint= torch.load('./checkpoint/encoder-epoch-0009-loss-49.5791.pth')
vqvae.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [8]:
encoder=vqvae.encoder.to('cuda')
vq_layer= vqvae.vq_layer.to('cuda')

In [11]:

list_batch=[]
count=0
for batch in dataloader:
    batch=batch.to('cuda')
    emb= encoder(batch)
    
    test,_= vq_layer(emb)
    list_batch.append(test)
    count+=1
    if count==2:
        break

In [12]:
list_batch[0].shape

torch.Size([2048, 16, 5, 5])

In [12]:
stacked_tensor = torch.cat(list_batch, dim=0)
stacked_tensor.shape

torch.Size([20, 3, 20, 20])

In [25]:
batch[0]

tensor([[[-0.4967, -0.4969, -0.4969,  ..., -0.4974, -0.4974, -0.4974],
         [-0.4967, -0.4969, -0.4969,  ..., -0.4974, -0.4974, -0.4974],
         [-0.4968, -0.4972, -0.4972,  ..., -0.4974, -0.4974, -0.4974],
         ...,
         [-0.4988, -0.4987, -0.4987,  ..., -0.4966, -0.4966, -0.4966],
         [-0.4990, -0.4987, -0.4987,  ..., -0.4966, -0.4966, -0.4966],
         [-0.4990, -0.4987, -0.4987,  ..., -0.4966, -0.4966, -0.4966]],

        [[-0.4993, -0.4993, -0.4993,  ..., -0.4993, -0.4993, -0.4993],
         [-0.4993, -0.4993, -0.4993,  ..., -0.4993, -0.4993, -0.4993],
         [-0.4993, -0.4993, -0.4993,  ..., -0.4992, -0.4992, -0.4992],
         ...,
         [-0.4996, -0.4996, -0.4996,  ..., -0.4988, -0.4988, -0.4988],
         [-0.4997, -0.4996, -0.4996,  ..., -0.4988, -0.4988, -0.4988],
         [-0.4997, -0.4996, -0.4996,  ..., -0.4988, -0.4988, -0.4988]],

        [[-0.4978, -0.4979, -0.4979,  ..., -0.4982, -0.4982, -0.4982],
         [-0.4978, -0.4979, -0.4979,  ..., -0

In [11]:
sum(batch[9]-batch[0])

tensor([[-0.4784, -0.5020, -0.5020, -0.4784, -0.5098, -0.5098, -0.4941, -0.5529,
         -0.5529, -0.5529, -0.5529, -0.6314, -0.6314, -0.6392, -0.6863, -0.6863,
         -0.7098, -0.7020, -0.7020, -0.6941],
        [-0.4784, -0.5020, -0.5020, -0.4784, -0.5098, -0.5098, -0.4941, -0.5529,
         -0.5529, -0.5529, -0.5529, -0.6314, -0.6314, -0.6392, -0.6863, -0.6863,
         -0.7098, -0.7020, -0.7020, -0.6941],
        [-0.4784, -0.4902, -0.4902, -0.4745, -0.4980, -0.4980, -0.4902, -0.5412,
         -0.5412, -0.5412, -0.5490, -0.6196, -0.6196, -0.6353, -0.6784, -0.6784,
         -0.7098, -0.6902, -0.6902, -0.6941],
        [-0.5490, -0.5961, -0.5961, -0.5412, -0.3961, -0.3961, -0.3686, -0.2980,
         -0.2980, -0.2980, -0.3059, -0.4275, -0.4275, -0.4667, -0.7059, -0.7059,
         -0.7294, -0.7765, -0.7765, -0.7686],
        [-0.5490, -0.5961, -0.5961, -0.5412, -0.3961, -0.3961, -0.3686, -0.2980,
         -0.2980, -0.2980, -0.3059, -0.4275, -0.4275, -0.4667, -0.7059, -0.7059,
      

In [16]:
encoder=vqvae.encoder.to('cuda')
vq_layer= vqvae.vq_layer.to('cuda')

In [18]:
vqvae.encoder(batch).view(-1, 16).shape

torch.Size([250, 16])

In [12]:
a,_=vqvae.vq_layer(vqvae.encoder(batch))

In [13]:
a.shape

torch.Size([10, 16, 5, 5])

In [35]:
sum(a[8]-a[0])

tensor([[-0.1426,  0.0590,  0.4325,  0.4278,  0.1018],
        [-0.0964,  0.0959,  0.5644,  0.3712,  0.1350],
        [-0.0614,  0.1468,  0.3577,  0.1026,  0.0429],
        [-0.0263,  0.0909,  0.0237, -0.3929, -0.2542],
        [-0.1569, -0.3325, -0.6611, -0.8779, -0.5176]], grad_fn=<AddBackward0>)

In [27]:
mlp = nn.Linear(16 * 5 * 5, 1024)
flattened_output = a.view(a.size(0), -1)

In [34]:
sum(flattened_output[9]-flattened_output[0])

tensor(2.4415, grad_fn=<AddBackward0>)

-----------------------------

In [ ]:
NAMES = ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']
dir = f'D:/data/crunch_large/data/'
anucleus={}
cell_id_group={}
import spatialdata as sd
for name in NAMES[1:3]:
    sdata = sd.read_zarr(f"{dir}/{name}.zarr")
    anucleus[name]=sdata['anucleus']
    cell_id_group[name]=sdata['cell_id-group']

E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModifi

In [ ]:
anucleus['DC5']

AnnData object with n_obs × n_vars = 140368 × 460
    obs: 'cell_id'
    var: 'gene_symbols'
    obsm: 'spatial'
    layers: 'counts'

In [73]:
name='DC5'
cell_id_train = cell_id_group[name].obs[cell_id_group[name].obs['group'] == 'train']['cell_id'].to_numpy()
cell_id_train = list(set(cell_id_train).intersection(set(anucleus[name].obs['cell_id'].unique())))

In [87]:
len(cell_id_train),cell_id_train[-1]

(140368, 224325)

In [88]:
cell_id_group[name].obs['cell_id'],cell_id_group[name].obs['group']

(5              6
 11            12
 13            14
 22            23
 36            37
            ...  
 223420    130831
 223433    130863
 223443    130886
 223450    130911
 223474    130959
 Name: cell_id, Length: 171019, dtype: uint32,
 5         train
 11        train
 13        train
 22        train
 36        train
           ...  
 223420     test
 223433     test
 223443     test
 223450     test
 223474     test
 Name: group, Length: 171019, dtype: object)

In [22]:
import pickle
pre_load_path= '../../pre_load'
name= 'DC1'
with open(f'{pre_load_path}/{name}_cells.pkl','rb') as f:
            cell_list= pickle.load(f)

In [23]:
for props in cell_list:
    props['anucleus']=np.array([])
with open(f'{pre_load_path}/{name}_cells.pkl','wb') as f:
            pickle.dump(cell_list,f)

In [20]:
len(cell_list),cell_list[-1]

(171019,
 {'center': (43763.45, 43448.01818181818),
  'cell_id': 224325,
  'label': 'train',
  'anucleus': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 2, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0

In [7]:
import spatialdata as sd

dir = f'D:/data/crunch_large/data/'

sdata = sd.read_zarr(f"{dir}/UC7_I.zarr")


E:\conda\envs\thitogene\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
import scanpy as sc
anucleus= sdata['anucleus']
layer_data = anucleus.layers['counts']

normalized_counts = layer_data / layer_data.sum(axis=1, keepdims=True) * 100

log_normalized_counts = np.log1p(normalized_counts)


AxisError: axis 1 is out of bounds for array of dimension 1

In [21]:
layer_data = anucleus.layers['counts'][0:1]
layer_data.shape
normalized_counts = layer_data / layer_data.sum(axis=1, keepdims=True) * 100

log_normalized_counts = np.log1p(normalized_counts)

In [11]:
log_normalized_counts[10],anucleus.X[10]

(array([1.50602482, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.01319454, 0.        , 1.83468451,
        1.50602482, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.50602482, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.01319454, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [1]:
import numpy as np
import torch

In [3]:
save_dir='D:/DATA//Gene_expression/Crunch/preprocessed/20'
groups =['train','test','validation']
NAMES = ['DC1','DC5', 'UC1_I', 'UC1_NI', 'UC6_I', 'UC6_NI', 'UC7_I', 'UC9_I']

for group in groups:
    for name in NAMES:
        tensors= torch.load(f'{save_dir}/{group}/{name}.pt')
        nps= tensors.cpu().numpy()
        np.save(f'{save_dir}/{group}/{name}.npy',nps)

In [2]:
%cd ..

C:\DATA\Crunch\neuron


In [3]:
from dataset import NeuronData

traindata= NeuronData(train=True, split =True)


ImportError: cannot import name 'NeuronData' from 'dataset' (C:\DATA\Crunch\neuron\preprocessed\dataset.py)

In [5]:
import numpy as np
a=np.array([1,2,3])


In [24]:
a=np.array([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), 
             (1, 0), (1, 2), (1, 3), (1, 4), (1, 5)])
a=a+1
a

array([[1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [1, 6],
       [2, 1],
       [2, 3],
       [2, 4],
       [2, 5],
       [2, 6]])